In [1]:
!pip install datasets

In [2]:
!pip install huggingface_hub

In [3]:
import datasets 
from datasets import load_dataset

In [7]:
dataset = load_dataset("nateraw/pascal-voc-2012", split = "train")

Using custom data configuration nateraw___pascal-voc-2012-c68607404d4811ac
Reusing dataset parquet (/aiffel/.cache/huggingface/datasets/parquet/nateraw___pascal-voc-2012-c68607404d4811ac/0.0.0/9296ce43568b20d72ff8ff8ecbc821a16b68e9b8b7058805ef11f06e035f911a)


In [8]:
print(dataset)

Dataset({
    features: ['image', 'mask'],
    num_rows: 2913
})


In [19]:
from torch import nn
from .WCC import WCC
from .QuantConv2d import QuantConv2d


def quantize_deeplabmobilev2(model, bit_w, bit_a):
    first_conv = model.backbone.low_level_features[0][0]
    last_layer = model.classifier.classifier[3]
    quantize_module(model, bit_w, bit_a)
    model.backbone.low_level_features[0][0] = first_conv
    model.classifier.classifier[3] = last_layer


def wavelet_deeplabmobilev2(model, levels, compress_rate, bit_w, bit_a):
    first_conv = model.backbone.low_level_features[0][0]
    last_layer = model.classifier.classifier[3]
    wavelet_module(model, levels, compress_rate, bit_w, bit_a)
    model.backbone.low_level_features[0][0] = first_conv
    model.classifier.classifier[3] = last_layer


def quantize_module(module, bit_w, bit_a):
    new_module = module
    if isinstance(module, nn.Conv2d):
        new_module = QuantConv2d(module.in_channels,
                                 module.out_channels,
                                 module.kernel_size,
                                 module.stride,
                                 module.padding,
                                 module.dilation,
                                 module.groups,
                                 module.bias is not None,
                                 bit_w,
                                 bit_a)
        new_module.weight = module.weight
        new_module.bias = module.bias
    for name, child in module.named_children():
        new_module.add_module(name, quantize_module(child, bit_w, bit_a))
    return new_module


def change_module_bits(module, bit_w, bit_a):
    if isinstance(module, QuantConv2d) or isinstance(module, WCC):
        module.change_bit(bit_w, bit_a)
    else:
        for name, child in module.named_children():
            change_module_bits(child, bit_w, bit_a)


def wavelet_module(module, levels, compress_rate, bit_w, bit_a):
    new_module = module
    if isinstance(module, nn.Conv2d):
        if module.kernel_size[0] > 1:
            new_module = QuantConv2d(module.in_channels,
                                     module.out_channels,
                                     module.kernel_size,
                                     module.stride,
                                     module.padding,
                                     module.dilation,
                                     module.groups,
                                     module.bias is not None,
                                     bit_w,
                                     bit_a)
            new_module.weight = module.weight
            new_module.bias = module.bias
        else:
            new_module = WCC(module.in_channels,
                             module.out_channels,
                             module.stride[0],
                             module.padding[0],
                             module.dilation[0],
                             module.groups,
                             module.bias is not None,
                             levels,
                             compress_rate,
                             bit_w,
                             bit_a)
            new_module.weight = nn.Parameter(module.weight.squeeze(-1))
            new_module.bias = module.bias
        if isinstance(module, QuantConv2d):
            new_module.act_quant.a_alpha = module.act_quant.a_alpha
            new_module.weight_quant.w_alpha = module.weight_quant.w_alpha
    else:
        for name, child in module.named_children():
            new_module.add_module(name, wavelet_module(child, levels, compress_rate, bit_w, bit_a))
    return new_module


def change_module_wt_params(module, compress_rate, levels):
    if isinstance(module, WCC):
        module.change_wt_params(compress_rate, levels)
    else:
        for name, child in module.named_children():
            change_module_wt_params(child, compress_rate, levels)

ImportError: attempted relative import with no known parent package

In [20]:
!pip install -U tensorboard


     |████████████████████████████████| 5.5 MB 6.5 MB/s            
     |████████████████████████████████| 311 kB 59.7 MB/s            
     |████████████████████████████████| 6.6 MB 65.6 MB/s            
     |████████████████████████████████| 5.3 MB 67.0 MB/s            
     |████████████████████████████████| 181 kB 99.5 MB/s            
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.3.3
    Uninstalling google-auth-2.3.3:
      Successfully uninstalled google-auth-2.3.3
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio

In [21]:
%load_ext tensorboard

In [22]:
import tensorflow as tf
import datetime


In [23]:
!rm -rf ./logs/


In [24]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/tensorboard/compat/__init__.py", line 42, in tf
    from tensorboard.compat import notf  # noqa: F401
ImportError: cannot import name 'notf' from 'tensorboard.compat' (/opt/conda/lib/python3.9/site-packages/tensorboard/compat/__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/opt/conda/lib/python3.9/site-packages/tensorboard/main.py", line 39, in run_main
    main_lib.global_init()
  File "/opt/conda/lib/python3.9/site-packages/tensorboard/main_lib.py", line 40, in global_init
    if getattr(tf, "__version__", "stub") == "stub":
  File "/opt/conda/lib/python3.9/site-packages/tensorboard/lazy.py", line 65, in __getattr__
    return getattr(load_once(self), attr_name)
  File "/opt